In [1]:
!pip install hyperopt

In [17]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from skimage import color, exposure

import os
import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

from sklearn.metrics import accuracy_score

%load_ext tensorboard

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [3]:
cd '/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_three/dw_matrix_road_sign'

/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_three/dw_matrix_road_sign


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim == 1 : y_train = to_categorical(y_train)
if y_test.ndim == 1 : y_test = to_categorical(y_test)

In [0]:
input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss = 'categorical_crossentropy', metrics = ['accuracy'], optimizer = 'Adam')

  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq = 1)

  model.fit(
      X_train,
      y_train,
      batch_size = params_fit.get('batch_size', 128),
      epochs = params_fit.get('epochs', 5),
      verbose = params_fit.get('verbose', 1),
      validation_data = params_fit.get('validation_data', (X_train, y_train)),
      callbacks = [tensorboard_callback]
  )

  return model

In [0]:
def predict(model_trained, X_test, y_test, scoring = accuracy_score):

  y_test_norm = np.argmax(y_test, axis = 1)

  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis = 1)

  return scoring(y_test_norm, y_pred)

In [0]:
def get_cnn_v5(input_shape, num_classes):
  return Sequential([
      Conv2D(filters = 32, kernel_size = (3, 3), activation = 'relu', input_shape = input_shape),
      Conv2D(filters = 32, kernel_size = (3, 3), activation = 'relu', padding = 'same'),
      MaxPool2D(),
      Dropout(0.3),

      Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu', padding = 'same'),
      Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu'),
      MaxPool2D(),
      Dropout(0.3),

      Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu', padding = 'same'),
      Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu'),
      MaxPool2D(),
      Dropout(0.3),


      Flatten(),

      Dense(1024, activation = 'relu'),
      Dropout(0.3),

      Dense(1024, activation = 'relu'),
      Dropout(0.3),

      Dense(num_classes, activation = 'softmax')
  ])

In [11]:
model = get_cnn_v5(input_shape, num_classes)
model_trained = train_model(model, X_train, y_train)
predict(model_trained, X_test, y_test)

Epoch 1/5
272/272 [==============================] - 13s 49ms/step - loss: 2.6073 - accuracy: 0.2656 - val_loss: 1.5319 - val_accuracy: 0.4642
Epoch 2/5
272/272 [==============================] - 12s 46ms/step - loss: 1.3454 - accuracy: 0.5468 - val_loss: 0.6865 - val_accuracy: 0.7597
Epoch 3/5
272/272 [==============================] - 12s 46ms/step - loss: 0.7229 - accuracy: 0.7642 - val_loss: 0.2527 - val_accuracy: 0.9187
Epoch 4/5
272/272 [==============================] - 12s 46ms/step - loss: 0.3968 - accuracy: 0.8751 - val_loss: 0.1108 - val_accuracy: 0.9676
Epoch 5/5
272/272 [==============================] - 12s 45ms/step - loss: 0.2761 - accuracy: 0.9144 - val_loss: 0.0586 - val_accuracy: 0.9849


0.9594104308390022

In [0]:
def get_model(params):
  return Sequential([
      Conv2D(filters = 32, kernel_size = (3, 3), activation = 'relu', input_shape = input_shape),
      Conv2D(filters = 32, kernel_size = (3, 3), activation = 'relu', padding = 'same'),
      MaxPool2D(),
      Dropout(params['dropout_cnn_block_one']),

      Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu', padding = 'same'),
      Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu'),
      MaxPool2D(),
      Dropout(params['dropout_cnn_block_two']),

      Conv2D(filters = 128, kernel_size = (3, 3), activation = 'relu', padding = 'same'),
      Conv2D(filters = 128, kernel_size = (3, 3), activation = 'relu'),
      MaxPool2D(),
      Dropout(params['dropout_cnn_block_three']),

      Flatten(),

      Dense(1024, activation = 'relu'),
      Dropout(params['dropout_dense_block_one']),

      Dense(1024, activation = 'relu'),
      Dropout(params['dropout_dense_block_two']),

      Dense(num_classes, activation = 'softmax')
  ])

In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ['accuracy'])

  model.fit (
      X_train,
      y_train,
      batch_size = int(params.get('batch_size', 128)),
      epochs = 5,
      verbose = 0
  )

  score = model.evaluate(X_test, y_test, verbose = 0)
  accuracy = score[1]
  print(params, 'accuracy = {}'.format(accuracy))

  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}

In [0]:
space = {
    'batch_size': hp.quniform('batch_size', 100, 200, 10),

    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0.3, 0.5),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0.3, 0.5),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0.3, 0.5),

    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one', 0.3, 0.7),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two', 0.3, 0.7),
}

In [37]:
best = fmin (
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

{'batch_size': 200.0, 'dropout_cnn_block_one': 0.30626135079769967, 'dropout_cnn_block_three': 0.3999049495608673, 'dropout_cnn_block_two': 0.34136101969172805, 'dropout_dense_block_one': 0.47373684626108403, 'dropout_dense_block_two': 0.6888043000364664}
accuracy = 0.9256235957145691
{'batch_size': 150.0, 'dropout_cnn_block_one': 0.3791001896728817, 'dropout_cnn_block_three': 0.4341218546112293, 'dropout_cnn_block_two': 0.3449143004709107, 'dropout_dense_block_one': 0.6712735388235765, 'dropout_dense_block_two': 0.6732895819158264}
accuracy = 0.8673469424247742
{'batch_size': 180.0, 'dropout_cnn_block_one': 0.41305268095607756, 'dropout_cnn_block_three': 0.4487589450818883, 'dropout_cnn_block_two': 0.38238572243823743, 'dropout_dense_block_one': 0.40923047121720213, 'dropout_dense_block_two': 0.5039791764397773}
accuracy = 0.9319728016853333
{'batch_size': 170.0, 'dropout_cnn_block_one': 0.4122751070314774, 'dropout_cnn_block_three': 0.4303625954806356, 'dropout_cnn_block_two': 0.3379